# Candidate Test 2022 Analysis Part 2

This exercise focuses on the candidate tests from two television networks: DR and TV2. Data from both tests have been given on a scale of five responses (-2, -1, 0, 1, 2). Consider normalizing the data or performing similar scaling transformations as needed.

---

There are 6 datasets included in this exercise:

- `alldata.xlsx`: Contains responses from both TV stations.
- `drdata.xlsx`: Contains responses from DR.
- `drq.xlsx`: Contains questions from DR.
- `tv2data.xlsx`: Contains responses from TV2.
- `tv2q.xlsx`: Contains questions from TV2.
- `electeddata.xlsx`: Contains responses from both TV stations for candidates who were elected to the parliament. Note that 9 members are missing; 7 of them didn't take any of the tests. Additionally, some notable figures like Mette F. and Lars Løkke did not participate in any of the tests.

---

It's entirely up to you how you approach this data, but at a *minimum*, your analysis should include:

- An analysis/description of which questions are most crucial concerning their placement on the axes.
- Average positions of parties concerning each question, preferably with accompanying plots of each (or selected) question.
- A clustering analysis where you attempt various cluster numbers, which would correspond to different parties. Discuss whether there is room for more clusters/parties or if a reduction is needed. Make sure you cover: **K-Means, Hierarchical clustering, and DBSCAN.**
- An overview of the political landscape of the elected candidates, highlighting which members agree or disagree the most and which parties or party members have significant disagreements.
- Feel free to explore further and remember that preprocessing, methodology, and evaluation metrics are not mentioned explicitly, but are implicitly assumed.

---

The following parties are represented:

| Party letter | Party name | Party name (English) | Political position |
| :-: | :-: | :-: | :-: |
| A | Socialdemokratiet | Social Democrats | Centre-left |
| V | Venstre | Danish Liberal Party | Centre-right |
| M | Moderaterne | Moderates | Centre-right |
| F | Socialistisk Folkeparti | Socialist People's Party | Left-wing |
| D | Danmarksdemokraterne | Denmark Democrats | Right-wing |
| I | Liberal Alliance | Liberal Alliance | Right-wing |
| C | Konservative | Conservative People's Party | Right-wing |
| Æ | Enhedslisten | Red-Green Alliance | Far-left |
| B | Radikale Venstre | Social Liberal Party | Centre-left |
| D | Nye Borgerlige | New Right | Far-right |
| Z | Alternativet | The Alternative | Centre-left |
| O | Dansk Folkeparti | Danish People's Party | Far-right |
| G | Frie Grønne | Free Greens | Centre-left |
| K | Kristendemokraterne | Christian Democrats | Centre-right |

Below you can see the results and the colors chosen to represent the parties. Use these colors in your analysis above.

![Alt text](image-1.png)


Others have undertaken similar analyses. You can draw inspiration from the following (use Google tranlsate if your Danish is rusty):

- [Analysis of where individual candidates stand relative to each other and their parties](https://v2022.dumdata.dk/)
- [Candidate Test 2022 – A deep dive into the data](https://kwedel.github.io/kandidattest2022/)
- [The Political Landscape 2019](https://kwedel.github.io/kandidattest2019/)



### Load all Data

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Import the reference data for parties
party_information = pd.read_csv('partyInformation.csv', header=0)

# Create a color key for parties based on the letter
color_key = party_information[['parti', 'farve']].set_index('parti').to_dict()['farve']
color_key_shorthand = party_information[['bogstav', 'farve']].set_index('bogstav').to_dict()['farve']
# partyInformation

In [12]:
# Reading all the data for candidates and their responses
data = pd.read_excel('alldata.xlsx', header=0)
# data

## Data Cleanup
The data cleanup undergoes the following steps:
- Correction of party names to match the reference table, some parties have longer official names that are not in the reference table.
    - Since most of the data already matches the reference table, we only need to correct the few that don't.
- Verification of duplicate candidate names.
- Removal of candidates where their age is set to 0.
    - This is obviously nonsensical and likely a placeholder for missing data, so we remove these candidates for simplicity. 


In [13]:
# Data Cleanup
# Correct party names to match the reference table, as a function for reusability
alternative_pary_names = {
    'Frie Grønne, Danmarks Nye Venstrefløjsparti': 'Frie Grønne',
    'Det Konservative Folkeparti': 'Konservative',
}
data['parti'] = data['parti'].replace(alternative_pary_names)

# Verify duplicate candidates
if data['navn'].duplicated().sum() > 0:
    print("Duplicate candidate names detected.")
else:
    print("No duplicate candidate names detected.")

data = data[data['alder'] > 0]

No duplicate candidate names detected.


In [14]:
# Extract info from every candidate and attach the party information for easy overview
# Also detaches the questions from the candidate data for easier analysis of the candidates themselves
candidateInfo = data.merge(party_information, on='parti', how='left')[['bogstav', 'navn', 'alder', 'storkreds', 'parti', 'holdning']]
# candidateInfo

In [15]:
# To sort all future charts by party, we can create a plot order based on the letters assigned to each party, UFG is removed and added at the end for unaligned candidates to show up last
plot_order = party_information['bogstav']
plot_order = plot_order[plot_order != 'UFG']
plot_order = plot_order.sort_values(ascending=True)
plot_order = pd.concat([plot_order, pd.Series('UFG')], ignore_index=True)
plot_order = plot_order.to_frame('bogstav').merge(party_information[['parti', 'bogstav']], on='bogstav', how='left')
# plot_order

alignment_order = {
    'Far-left': -3,
    'Left-wing': -2,
    'Centre-left': -1,
    'Non-Aligned': 0,
    'Centre-right': 1,
    'Right-wing': 2,
    'Far-right': 3
}